<h1 style="text-align:center;">Setting an ISCSI boot instance for HPE iLO systems</h1>

<div class="alert alert-block alert-info" style="border-radius: 20px;">
 Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>set an ISCSI boot instance for HPE iLO systems.</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "#" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    Also do run each succesive cell of code 
    
</div>

<div class="alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary modules from the ilorest library.</h3><br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
        
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
from get_resource_directory import get_resource_directory
import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. Below is a set_bios_iscsi that sets the ISCSI boot instance for HPE iLO systems.</h3><br>
    There is a hierarchy involved in iLO Rest Api and everything is in form of <a href = "https://www.w3schools.com/whatis/whatis_json.asp" style="background-color:powderblue;">JSON</a> objects i.e. name-value pairs. So we are simply following the hierarchy to navigate to required information.

1. The function takes two parameters: <b>_redfishobj</b> (a Redfish client object) and <b>iscsi_properties</b> (a dictionary containing the iSCSI properties to be set).<br><br>
2. It calls the <b>get_resource_directory function</b> to obtain the resource directory from the Redfish service. If <b>DISABLE_RESOURCE_DIR</b> is set to True or the resource directory is empty, the code retrieves the necessary URIs manually.<br><br>
3. <b>If the resource directory is disabled or not available</b>, the code fetches the systems URI and retrieves the <b>BIOS URI</b> from the response. The code retrieves the <b>iSCSI URI</b> from the BIOS response and fetches the <b>iSCSI data</b>.<br><br>
4. <b>If the resource directory is enabled and available</b>, the code iterates over the instances in the resource directory and searches for the iSCSI instance based on its type.<br><br>
5. <b>If the iSCSI data is found</b>, the code loops over the iSCSI sources and checks if the iSCSIBootInstance property matches the current index. If a match is found, the code updates the iSCSI properties for that instance.<br><br>
6. The code sends a <b>PATCH</b> request to the iSCSI URI with the updated iSCSI sources data.<br><br>
7. If the response status is 400, indicating an error, the code attempts to print the error message. If the response status is not 200 or 201, an error message is printed.<br><br>
8. If the response status is 200, the code indicates success and suggests that a system reboot will be required to apply the changes. It also prints the response details.<br><br>

The loop is broken after processing the desired iSCSI instance.
<br>
    
</div>
 



In [ ]:
def set_bios_iscsi(_redfishobj, iscsi_properties):

    iscsi_uri = None
    iscsi_data = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        systems_members_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_members_response = _redfishobj.get(systems_members_uri)
        bios_uri = systems_members_response.obj['Bios']['@odata.id']
        bios_response = _redfishobj.get(bios_uri)
        iscsi_uri = bios_response.obj.Oem.Hpe.Links['iScsi']['@odata.id']
        iscsi_data = _redfishobj.get(iscsi_uri)
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#HpeiSCSISoftwareInitiator.' in instance['@odata.type']:
                iscsi_uri = instance['@odata.id']
                iscsi_data = _redfishobj.get(iscsi_uri)

    if iscsi_data:
        for indx, inst in enumerate(iscsi_data.obj['iSCSISources']):
            if iscsi_properties['iSCSIBootInstance'] == indx:
                iscsi_data.dict['iSCSISources'][indx].update(iscsi_properties)
                resp = _redfishobj.patch(iscsi_uri, {'iSCSISources' : iscsi_data.dict['iSCSISources']})
                #If iLO responds with soemthing outside of 200 or 201 then lets check the
                #iLO extended info error message to see what went wrong
                if resp.status == 400:
                    try:
                        print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], \
                                                                        indent=4, sort_keys=True))
                    except Exception as excp:
                        print(json.dumps(resp.ori, indent=4, sort_keys=True))
                        sys.stderr.write("A response error occurred, unable to access iLO " \
                                         "Extended Message Info...")
                elif resp.status != 200:
                    sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
                else:
                    print("Success! A system reboot will be required to complete the change.\n")
                    print(json.dumps(resp.dict, indent=4, sort_keys=True))
                break


<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is Main function overview.<br></h3>
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE.<br>
    <ul>
        <li>Once you enter everything a redfish object is created with the help of RedfishClient class and a session is created and now you will be logged in to the server</li>
        <li>We are passing the required information to change the password of iLO user account i.e. base_url, username, password</li>
        <li>If server is down or Unreachable we get server not reachable message.</li>
        <li>The set_bios_iscsi function is called, passing the RedfishClient object (REDFISHOBJ) and the iSCSI properties dictionary (ISCSI_PROPERTIES).</li>

<li>After setting the iSCSI properties, the logout method of the RedfishClient object is called to end the session with the server.</li>
    </ul>
</div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [ ]:
if __name__ == "__main__":
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Name: ")
    LOGIN_PASSWORD =getpass.getpass("Enter password: ")
    
    #iscsi properties
    ISCSI_PROPERTIES = {"iSCSIBootInstance": 2, \
                      "iSCSITargetName": "Target", \
                      "iSCSIAttemptName": "Empty", \
                      "iSCSIConnectRetry": 5}
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    set_bios_iscsi(REDFISHOBJ, ISCSI_PROPERTIES)
    REDFISHOBJ.logout()


<div class = "alert alert-block alert-success" style="border-radius: 20px;">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>